## 1. Load the needed modules
Run this. All modules should be installed (if not, install all packages from requirments.txt)

In [1]:

from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip



## 2. Create a local Spark-session
Several things can go wrong:
If error on build.getOrCreate():
    Check your environment variabels SPARK_HOME and PATH. They have to refer to the spark installation
If error on configure_spark_with_delta_pip
    Problem is related to the delta-spark installation

In [1]:
def LDA_coefficients(X,lda):
    nb_col = X.shape[1]
    matrix= np.zeros((nb_col+1,nb_col), dtype=int)
    Z=pd.DataFrame(data=matrix,columns=X.columns)
    for j in range(0,nb_col):
        Z.iloc[j,j] = 1
    LD = lda.transform(Z)
    nb_funct= LD.shape[1]
    resultaat = pd.DataFrame();
    index = ['const']
    for j in range(0,LD.shape[0]-1):
        index = np.append(index,'C'+str(j+1))
    for i in range(0,LD.shape[1]):
        coef = [LD[-1][i]]
        for j in range(0,LD.shape[0]-1):
            coef = np.append(coef,LD[j][i]-LD[-1][i])
        result = pd.Series(coef)
        result.index = index
        column_name = 'LD' + str(i+1)
        resultaat[column_name] = result
    return resultaat

In [3]:

builder = SparkSession.builder \
    .appName("DimDate") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.shuffle.partitions", "4") \
    .master("local[*]")

#When shuffled 4 concurrent tasks will be created.
builder = configure_spark_with_delta_pip(builder)
spark = builder.getOrCreate()
builder.getOrCreate()

## 3. Reading txt file  with spark-sql
If this runs, Spark is working correctly

In [14]:
from pyspark.sql.functions import *

# File location and type
file_location = "./FileStore/tables/shakespeare.txt"
file_type = "text"

df = spark.read.format(file_type)  \
  .load(file_location)

words = df.selectExpr("explode(split(value, ' '))").filter("col != ''")
words.createOrReplaceTempView("words")
lowerwords = spark.sql("select lower(col), count(*) as word from words group by lower(col) order by word desc limit 20")
lowerwords.show()

AnalysisException: cannot resolve 'split(split(value, ',', -1), ' ', -1)' due to data type mismatch: argument 1 requires string type, however, 'split(value, ',', -1)' is of array<string> type.; line 1 pos 8;
'Project [unresolvedalias('explode(split(split(value#1230, ,, -1),  , -1)), Some(org.apache.spark.sql.Column$$Lambda$2112/0x0000000101114840@5f49a810))]
+- Relation [value#1230] text


## 4. Writing to Delta-table


In [12]:
wordCounts = spark.sql("select lower(col) as word, count(*) as amount from words group by lower(col) order by word")
wordCounts.write.format("delta").mode("overwrite").saveAsTable("wordCounts")


# TESTSTUFF


In [16]:
from delta import DeltaTable

DeltaTable.create(spark) \
  .tableName("default.people10m") \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING", comment = "surname") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("dateOfBirth", DateType(), generatedAlwaysAs="CAST(birthDate AS DATE)") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .execute()

ParseException: 
Syntax error at or near 'GENERATED'(line 1, pos 43)

== SQL ==
CREATE OR REPLACE TABLE demo (   id BIGINT GENERATED ALWAYS AS IDENTITY,   product_type STRING,   sales BIGINT) );
-------------------------------------------^^^


## Step 2: Create all typical dimension fields
Because we want to represent the date in different ways (weekday, month...), we have to do several tranformations. You can see an extract of our go
```
+------+--------+------------+------------+-------------+-----------+-----------+---------+--------------------+---------+----------+----------------+---------+-------------+-------------+
|dateSK| dateInt|CalendarDate|CalendarYear|CalendarMonth|MonthOfYear|CalendarDay|DayOfWeek|DayOfWeekStartMonday|IsWeekDay|DayOfMonth|IsLastDayOfMonth|DayOfYear|WeekOfYearIso|QuarterOfYear|
+------+--------+------------+------------+-------------+-----------+-----------+---------+--------------------+---------+----------+----------------+---------+-------------+-------------+
|     0|20090101|  2009-01-01|        2009|      January|          1|   Thursday|        5|                   4|        Y|         1|               N|        1|            1|            1|
|     1|20090102|  2009-01-02|        2009|      January|          1|     Friday|        6|                   5|        Y|         2|               N|        2|            1|            1|
|     2|20090103|  2009-01-03|        2009|      January|          1|   Saturday|        7|                   6|        N|         3|               N|        3|            1|            1|
|     3|20090104|  2009-01-04|        2009|      January|          1|     Sunday|        1|                   7|        N|         4|               N|        4|            1|            1|
```

### Method a: Use spark.sql to perform all the transformations with the help of a sql-query.
For many, creating an SQL-select statement is the most easy way to perform the transformation.

In [4]:
dimDate = spark.sql("select dateSK, \
  year(calendarDate) * 10000 + month(calendarDate) * 100 + day(calendarDate) as dateInt, \
  CalendarDate, \
  year(calendarDate) AS CalendarYear, \
  date_format(calendarDate, 'MMMM') as CalendarMonth, \
  month(calendarDate) as MonthOfYear, \
  date_format(calendarDate, 'EEEE') as CalendarDay, \
  dayofweek(calendarDate) AS DayOfWeek, \
  weekday(calendarDate) + 1 as DayOfWeekStartMonday, \
  case \
    when weekday(calendarDate) < 5 then 'Y' \
    else 'N' \
  end as IsWeekDay, \
  dayofmonth(calendarDate) as DayOfMonth, \
  case \
    when calendarDate = last_day(calendarDate) then 'Y' \
    else 'N' \
  end as IsLastDayOfMonth, \
  dayofyear(calendarDate) as DayOfYear, \
  weekofyear(calendarDate) as WeekOfYearIso, \
  quarter(calendarDate) as QuarterOfYear \
from  \
  neededDates \
order by \
  calendarDate")

dimDate.show()

+------+--------+------------+------------+-------------+-----------+-----------+---------+--------------------+---------+----------+----------------+---------+-------------+-------------+
|dateSK| dateInt|CalendarDate|CalendarYear|CalendarMonth|MonthOfYear|CalendarDay|DayOfWeek|DayOfWeekStartMonday|IsWeekDay|DayOfMonth|IsLastDayOfMonth|DayOfYear|WeekOfYearIso|QuarterOfYear|
+------+--------+------------+------------+-------------+-----------+-----------+---------+--------------------+---------+----------+----------------+---------+-------------+-------------+
|     0|20090101|  2009-01-01|        2009|      January|          1|   Thursday|        5|                   4|        Y|         1|               N|        1|            1|            1|
|     1|20090102|  2009-01-02|        2009|      January|          1|     Friday|        6|                   5|        Y|         2|               N|        2|            1|            1|
|     2|20090103|  2009-01-03|        2009|      Januar

* ```spark.sql``` is used to create the select query and returns the desired DataFrame. Remember to look-up the possible functions [here](https://spark.apache.org/docs/latest/api/sql/).
* ```dimDate.show()``` s used to show the records in a DataFrame. Use it during development, but disable when not needed anymore


### Method b: Use the dataframe API

This method does not use the sql-like language. You can achieve the same with this method and you get better code completion. See [DataFrame API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis)
As an example two columns where added with   ```withColumn``` .
```èxpr()``` is used to write a snippet of 'sql' code and parse it into a column.

In [5]:
#from pyspark.sql.functions import explode, expr, sequence,col, date_format
df_SparkSQL = df_SQL \
    .withColumn("year", date_format(col("calendarDate"),'yyyy')) \
    .withColumn("month", date_format(col("calendarDate"),'MMMM')) \
    .withColumn("lasyDayOfMonth" \
                ,expr("case when calendarDate = last_day(calendarDate) then 'Y' \
                else 'N' \
                end as IsLastDayOfMonth"))
df_SparkSQL.show()

+------------+------+----+-------+--------------+
|calendarDate|dateSK|year|  month|lasyDayOfMonth|
+------------+------+----+-------+--------------+
|  2009-01-01|     0|2009|January|             N|
|  2009-01-02|     1|2009|January|             N|
|  2009-01-03|     2|2009|January|             N|
|  2009-01-04|     3|2009|January|             N|
|  2009-01-05|     4|2009|January|             N|
|  2009-01-06|     5|2009|January|             N|
|  2009-01-07|     6|2009|January|             N|
|  2009-01-08|     7|2009|January|             N|
|  2009-01-09|     8|2009|January|             N|
|  2009-01-10|     9|2009|January|             N|
|  2009-01-11|    10|2009|January|             N|
|  2009-01-12|    11|2009|January|             N|
|  2009-01-13|    12|2009|January|             N|
|  2009-01-14|    13|2009|January|             N|
|  2009-01-15|    14|2009|January|             N|
|  2009-01-16|    15|2009|January|             N|
|  2009-01-17|    16|2009|January|             N|


> ## TASK:
> Complete the transformation in method b until the result matches the result of method a.

# Step 3: Writing the data to a delta-file

In [7]:
spark.stop()